In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output, display
sns.set()

In [2]:
#Import client_id, client_secret, etc from external .json file (to keep user details out of the .ipynb file).
df = pd.read_json("/Users/arnavprasad/Google Drive/Documents/Coding/Spotify/auth_details.json", typ="series")

username = "arnav7"
client_id = df["client_id"]
client_secret = df["client_secret"]
redirect_uri = df["redirect_uri"]

In [3]:
scope = "user-top-read playlist-modify-public" #define your desired goal(s)

In [4]:
token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [5]:
sp = spotipy.Spotify(auth=token)

In [6]:
features = ["danceability", "energy", "loudness", "acousticness", "instrumentalness", "liveness", "tempo"]

#Function to return a simplified dictionary with only the required data

def track_info(song_dict, audio_features=False):
    """
    Returns all the necessary information about a song and removes the rest.

    Input:
    song_dict: a dictionary that contains all the information about a song (not all of which is necessary for our purposes).

    Outputs:
    track_info: simplified dictionary, with keys
    - track_name
    - track_id
    - artist
    - album
    - duration
    - popularity
    - if audio_features==True, also returns the song's "audio features" as defined by Spotify
    """

    track_data = {}

    track_data["track_name"] = song_dict["name"]
    track_data["artist"] = song_dict["artists"][0]["name"]
    track_data["album"] = song_dict["album"]["name"]
    track_data["track_id"] = song_dict["id"]
    track_data["duration_ms"] = song_dict["duration_ms"]
    track_data["popularity"] = song_dict["popularity"]

    if audio_features == True:
        for feature in features:
            track_data[feature] = sp.audio_features(tracks=[song_dict["id"]])[0][feature]
        
    return track_data


#Function to display search results
def search_results(query, limit=5, audio_features=False):
    """
    Organise and display search results.
    """
    results = sp.search(query, limit=limit)["tracks"]["items"] #list of all the search results, as dictionaries

    search_rows = []

    for track in results:
        search_rows.append(track_info(track, audio_features=audio_features)) #append only the required information

    search_df = pd.DataFrame.from_dict(search_rows)

    return search_df



In [7]:
#Print out your top songs over three different time periods

ranges = ["short_term", "medium_term", "long_term"]

for sp_range in ranges:
    print(sp_range + "\n")
    results = sp.current_user_top_tracks(time_range=sp_range, limit=10)
    for i, item in enumerate(results["items"]):
        print(str(i+1) + " " + item["name"] + " // " + item["artists"][0]["name"])
    print("\n")


short_term

1 Old Fashioned // Bruno Major
2 Photo ID // Remi Wolf
3 Karma Police // Scary Pockets
4 Cheesin' // Cautious Clay
5 New York, New York (Live) // Jon Batiste
6 Does It Make You Feel Good? // Joesef
7 Out Of My Head // Mac DeMarco
8 With Love To An Ex (feat. Moonchild Sanelly) // Gorillaz
9 Dang! (feat. Anderson .Paak) // Mac Miller
10 Vacation // Still Woozy


medium_term

1 Herside Story // GoldLink
2 Tribe (with J. Cole) // Bas
3 Out Of My Head // Mac DeMarco
4 If You’re Too Shy (Let Me Know) // The 1975
5 Sunblind // Fleet Foxes
6 The Tracks Of My Tears // Smokey Robinson & The Miracles
7 Great Dane - Radio Edit // Cosmo Pyke
8 The Recipe // SiR
9 CUT EM IN (feat. Rick Ross) // Anderson .Paak
10 Baguetti (with JID & Kenny Beats) // Smino


long_term

1 Freaking Out the Neighborhood // Mac DeMarco
2 SUGAR // BROCKHAMPTON
3 Waitin' On The Day // John Mayer
4 If I Ever Get Around To Living // John Mayer
5 Still Beating // Mac DeMarco
6 My 1-Bedroom // Theo Katzman
7 What Di

In [9]:
#Search for seed songs

#if you want to include the seed songs in the playlist
include_seeds = False

seeds = []

seed_rows = []

n = int(input("How many songs do you want in your playlist?"))

#Generate seed list
for i in range(5):

    if i == 0:
        search_term = input("First seed track: ")
        
        results = search_results(search_term)
        display(results[["track_name", "artist", "album"]]) #display concise version of search results

        index = int(input("Select the index of the track you want (indexing starts from zero): ")) #choose which song you want

        track = results.loc[index]

        clear_output()

        seeds.append(track["track_id"])
        seed_rows.append(track)
    
    else:
        cont = int(input("Enter 1 if you want to continue, and 0 if not: "))

        if cont == 1:

            search_term = input("Next seed track: ")

            results = search_results(search_term)
            display(results[["track_name", "artist", "album"]])

            index = int(input("Select the index of the track you want (indexing starts from zero): "))

            track = results.loc[index]

            clear_output()

            seeds.append(track["track_id"])
            seed_rows.append(track)
        else:
            break

seed_songs = pd.DataFrame.from_dict(seed_rows) #DataFrame for seed songs

results = sp.recommendations(seed_tracks=seeds, limit=n)["tracks"]
rec_rows = []

for track in results:
    rec_rows.append(track_info(track))

search_recs = pd.DataFrame.from_dict(rec_rows) #DataFrame for recommendation playlist


if include_seeds==True:
    search_recs = seed_songs.append(search_recs, ignore_index=True)

In [10]:
#Check to verify your seed songs
seed_songs[["track_name", "artist", "album", "popularity"]]

,track_name,artist,album,popularity
0,I Wish,Tom Misch,I Wish,55
0,In My Room,Frank Ocean,In My Room,79
0,Cory Wong,Vulfpeck,The Beautiful Game,52
0,If Only,Raveena,If Only,61
0,Cómo Me Quieres,Khruangbin,Con Todo El Mundo,54


In [11]:
#Look at the generated playlist
search_recs[["track_name", "artist", "album", "popularity"]]

,track_name,artist,album,popularity
0,By Any Means,Jorja Smith,By Any Means,69
1,Hurry,Teyana Taylor,K.T.S.E.,53
2,Tee Time,Vulfpeck,Mr Finish Line,42
3,Reason,Hablot Brown,Soulection Black Label: Hablot Brown,46
4,Green & Gold,Lianne La Havas,Blood,64
5,Only Memories Remain,My Morning Jacket,The Waterfall,44
6,Judas,Esperanza Spalding,Emily’s D+Evolution,42
7,Prickly Pear,Portico Quartet,Knee-Deep in the North Sea,61
8,Long Time - Intro,Playboi Carti,Die Lit,70
9,Vex Oh (feat. Eight9FLY),KAYTRANADA,BUBBA,66


In [12]:
#Create list with only the track ids
rec_track_ids = list(search_recs["track_id"])

In [13]:
#The URI for the playlist
playlist_id = "3pRiSIEh2iQEihpRoTmbDL"

#Make sure you don't add any duplicates

playlist_tracks = [] #tracks already in playlist

playlist_items = sp.playlist_tracks(playlist_id=playlist_id)["items"]

for item in playlist_items:
    playlist_tracks.append(item["track"]["id"]) #list of all the track ids in the playlist

for rec_track in rec_track_ids:
    if rec_track in playlist_tracks:
        rec_track_ids.remove(rec_track)

In [14]:
sp.user_playlist_add_tracks(user=username, playlist_id=playlist_id, tracks=rec_track_ids)

{'snapshot_id': 'OTMsNThkYjcwODFjZTRjZDMwYzFkNjMxMWIxY2QwYTVmMmJiZWYyODYxYQ=='}